In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
import numpy as np
from collections import Counter
from tqdm import tqdm
import torchvision.models as models
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.nn.functional as F
import torch.optim as optim
from torchvision.models import convnext_tiny
from sklearn.model_selection import train_test_split
import timm

In [2]:
SEED = 42 
np.random.seed(SEED) 
torch.manual_seed(SEED)

In [3]:
class DERMNET_DATASET(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        super().__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform

        self.labels_dict = {
            'Light Diseases and Disorders of Pigmentation': 0, 
            'Lupus and other Connective Tissue diseases': 1,
            'Acne and Rosacea Photos': 2,
            'Systemic Disease': 3,
            'Poison Ivy Photos and other Contact Dermatitis': 4,
            'Vascular Tumors': 5,
            'Urticaria Hives': 6,
            'Atopic Dermatitis Photos': 7,
            'Bullous Disease Photos': 8,
            'Hair Loss Photos Alopecia and other Hair Diseases': 9,
            'Tinea Ringworm Candidiasis and other Fungal Infections': 10,
            'Psoriasis pictures Lichen Planus and related diseases': 11,
            'Melanoma Skin Cancer Nevi and Moles': 12,
            'Nail Fungus and other Nail Disease': 13,
            'Scabies Lyme Disease and other Infestations and Bites': 14,
            'Eczema Photos': 15,
            'Exanthems and Drug Eruptions': 16,
            'Herpes HPV and other STDs Photos': 17,
            'Seborrheic Keratoses and other Benign Tumors': 18,
            'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 19,
            'Vasculitis Photos': 20,
            'Cellulitis Impetigo and other Bacterial Infections': 21,
            'Warts Molluscum and other Viral Infections': 22
        }

        dataset_type = 'train' if self.train else 'test'
        dataset_dir = os.path.join(root_dir, dataset_type)

        # Tạo một dictionary để lưu danh sách ảnh cho từng nhãn
        label_to_images = {label: [] for label in range(len(self.labels_dict))}

        # Duyệt qua các thư mục con và lưu đường dẫn ảnh theo nhãn tương ứng
        for class_name, class_idx in self.labels_dict.items():
            class_folder = os.path.join(dataset_dir, class_name)
            if os.path.isdir(class_folder):
                image_paths = sorted([os.path.join(class_folder, img) 
                                      for img in os.listdir(class_folder) 
                                      if img.endswith(('.png', '.jpg', '.jpeg'))])
                label_to_images[class_idx].extend(image_paths)

        # Tạo danh sách ảnh và nhãn xen kẽ
        self.image_paths = []
        self.labels = []
        for label, images in label_to_images.items():
            for img in images:
                self.image_paths.append(img)
                self.labels.append(label)

    def __len__(self):
        return len(self.labels)  # Tổng số lượng ảnh

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Mở ảnh và áp dụng transform nếu có
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)

        return image, label

    def show_image(self, idx):
        image, label = self.__getitem__(idx)
        keys = [k for k, v in self.labels_dict.items() if v == label]
        
        # Hiển thị ảnh
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Label: {keys[0]}")
        plt.show()


In [4]:
from torch.utils.data import Dataset, Subset

class CustomSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = Subset(dataset, indices)
        self.indices = indices
        self.labels_dict = dataset.labels_dict  # Giữ nguyên labels_dict từ dataset gốc
        self.dataset_cls = dataset  # Duy trì tham chiếu tới dataset gốc

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        return self.dataset[idx]

    def get_class_counts(self):
        """Tính toán số lượng ảnh trong mỗi lớp."""
        class_counts = {class_name: 0 for class_name in self.labels_dict.keys()}
        for idx in self.indices:
            label = self.dataset_cls.labels[idx]
            class_name = [k for k, v in self.labels_dict.items() if v == label][0]
            class_counts[class_name] += 1
        return class_counts

    def show_image(self, idx):
        """Hiển thị ảnh."""
        actual_idx = self.indices[idx]
        self.dataset_cls.show_image(actual_idx)

In [5]:
import torchvision.transforms as transforms 
# Định nghĩa các bước tiền xử lý 
transform = transforms.Compose([ transforms.Resize((128, 128)),   
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [6]:
train_dataset = DERMNET_DATASET('/kaggle/input/dermnet', train=True, transform=transform)

In [7]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate loss and accuracy
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples
    
    return epoch_loss, epoch_accuracy

def evaluate(model, val_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_targets = []
    all_preds = []
    top_5_correct = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Calculate loss
            running_loss += loss.item() * images.size(0)
            
            # Calculate Top-1 accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            # Collect predictions and targets for F1, Precision, Recall calculation
            all_targets.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            


    # Calculate average loss and top-1 accuracy
    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    # Calculate F1 Score, Precision, and Recall
    f1 = f1_score(all_targets, all_preds, average="weighted")
    precision = precision_score(all_targets, all_preds, average="weighted")
    recall = recall_score(all_targets, all_preds, average="weighted")
    
    return epoch_loss, epoch_accuracy, f1, precision, recall


In [8]:
from torchvision.models import convnext_tiny, densenet121, mobilenet_v2, resnet50
import timm

In [9]:
class ConvNeXtTinyModel(nn.Module):
    def __init__(self, num_classes=23, pretrained=True, frozen=False):
        super(ConvNeXtTinyModel, self).__init__()
        
        # Load pre-trained ConvNeXt Tiny model
        self.model = convnext_tiny(pretrained=pretrained)
        
        # Optionally freeze backbone layers
        if frozen:
            for param in self.model.features.parameters():
                param.requires_grad = False
        
        # Replace the classifier layer (fully connected layer) to match the number of classes
        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        return self.model(x)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
num_epochs = 20
batch_size = 128

In [13]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Subset
import os

num_epochs = 40
batch_size = 128

# Đường dẫn lưu các mô hình tốt nhất cho mỗi fold
output_dir = "./convnext_frozen"
os.makedirs(output_dir, exist_ok=True)

# StratifiedKFold cross-validator với số lượng fold là 5
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)

# Lấy danh sách các chỉ số của dataset
dataset_indices = np.arange(len(train_dataset))
labels = train_dataset.labels  # Giả định nhãn đã được gán trong dataset

# Biến lưu kết quả tốt nhất của từng fold
best_accuracies = []

# Bắt đầu vòng lặp qua từng fold
for fold, (train_indices, val_indices) in enumerate(skf.split(dataset_indices, labels)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Kiểm tra phân phối nhãn trong mỗi fold
    train_label_distribution = np.bincount([labels[i] for i in train_indices], minlength=len(np.unique(labels))) / len(train_indices)
    val_label_distribution = np.bincount([labels[i] for i in val_indices], minlength=len(np.unique(labels))) / len(val_indices)
    print(f"Train label distribution for fold {fold + 1}: {train_label_distribution}")
    print(f"Validation label distribution for fold {fold + 1}: {val_label_distribution}")

    # Tạo subset cho train và validation
    train_subset = Subset(train_dataset, train_indices)
    val_subset = Subset(train_dataset, val_indices)

    # Sử dụng WeightedRandomSampler để cân bằng nhãn cho tập train
    train_labels = [train_dataset.labels[i] for i in train_indices]
    class_sample_counts = np.bincount(train_labels)
    weights = 1. / class_sample_counts[train_labels]
    train_sampler = WeightedRandomSampler(weights, len(weights))

    # Tạo DataLoader cho train và validation
    train_dataloader = DataLoader(train_subset, batch_size=batch_size, sampler=train_sampler)
    val_dataloader = DataLoader(val_subset, batch_size=batch_size)

    # Khởi tạo model, optimizer và criterion
    model = ConvNeXtTinyModel(num_classes=23, frozen=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Biến lưu độ chính xác tốt nhất cho fold hiện tại
    best_val_accuracy = 0.0

    # Huấn luyện qua các epoch
    for epoch in range(num_epochs):
        print(f"Epoch [{epoch + 1}/{num_epochs}] - Fold {fold + 1}")

        # Huấn luyện mô hình
        train_loss, train_accuracy = train(model, train_dataloader, criterion, optimizer, device)

        # Đánh giá mô hình trên tập validation
        val_loss, val_accuracy, f1, precision, recall = evaluate(model, val_dataloader, criterion, device)

        # Lưu model nếu đạt độ chính xác tốt nhất
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            model_path = os.path.join(output_dir, f"best_model_fold_{fold + 1}.pth")
            torch.save(model.state_dict(), model_path)
            print(f"Checkpoint saved for fold {fold + 1}")

        # In thông tin cho mỗi epoch
        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
        print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

    # Lưu kết quả tốt nhất cho fold hiện tại
    best_accuracies.append(best_val_accuracy)


Fold 1/5
Train label distribution for fold 1: [0.03648051 0.02699879 0.05399759 0.03897147 0.01671354 0.03093612
 0.0136601  0.03141824 0.02884693 0.01534753 0.0835677  0.0903174
 0.02973082 0.06685416 0.02772198 0.07938931 0.0259542  0.02603455
 0.08814785 0.07384492 0.02675773 0.01848132 0.06982724]
Validation label distribution for fold 1: [0.03663239 0.02699229 0.05398458 0.03888175 0.01670951 0.03116967
 0.01349614 0.031491   0.02859897 0.01542416 0.08354756 0.09029563
 0.02988432 0.06683805 0.02763496 0.07937018 0.02602828 0.02602828
 0.08804627 0.07390746 0.02667095 0.01863753 0.06973008]


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 207MB/s]  


Epoch [1/20] - Fold 1


Checkpoint saved for fold 1
Epoch 1, Train Loss: 2.5958, Train Acc: 0.2455
Val Loss: 2.4739, Val Acc: 0.2812
F1 Score: 0.2750, Precision: 0.3268, Recall: 0.2812
Epoch [2/20] - Fold 1


Checkpoint saved for fold 1
Epoch 2, Train Loss: 2.2338, Train Acc: 0.3514
Val Loss: 2.3299, Val Acc: 0.3188
F1 Score: 0.3168, Precision: 0.3593, Recall: 0.3188
Epoch [3/20] - Fold 1


...

Checkpoint saved for fold 5
Epoch 16, Train Loss: 1.5936, Train Acc: 0.5288
Val Loss: 2.0859, Val Acc: 0.3934
F1 Score: 0.3963, Precision: 0.4350, Recall: 0.3934
Epoch [17/20] - Fold 5


Epoch 17, Train Loss: 1.5738, Train Acc: 0.5337
Val Loss: 2.0920, Val Acc: 0.3918
F1 Score: 0.3938, Precision: 0.4327, Recall: 0.3918
Epoch [18/20] - Fold 5


Epoch 18, Train Loss: 1.6016, Train Acc: 0.5241
Val Loss: 2.1044, Val Acc: 0.3851
F1 Score: 0.3918, Precision: 0.4321, Recall: 0.3851
Epoch [19/20] - Fold 5


Epoch 19, Train Loss: 1.5691, Train Acc: 0.5394
Val Loss: 2.0837, Val Acc: 0.3893
F1 Score: 0.3933, Precision: 0.4292, Recall: 0.3893
Epoch [20/20] - Fold 5


Checkpoint saved for fold 5
Epoch 20, Train Loss: 1.5569, Train Acc: 0.5415
Val Loss: 2.0817, Val Acc: 0.3938
F1 Score: 0.3955, Precision: 0.4248, Recall: 0.3938
